In [1]:
import pandas as pd
data = pd.read_csv("./aos_data/merged_phuc_data.csv")  # Assuming you save your data as CSV


In [ ]:

from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Example: Normalize raw features
scaler = MinMaxScaler()
features = ['AccelX', 'AccelY', 'AccelZ', 'GyroX', 'GyroY', 'GyroZ', 'IR']
data[features] = scaler.fit_transform(data[features])

# Windowing (simplified, assumes 5 samples per window)
window_size = 5
X, y = [], []
for i in range(0, len(data) - window_size, window_size):
    window = data.iloc[i:i+window_size][features].mean().values  # Mean of window
    label = data.iloc[i:i+window_size]['Label'].mode()[0]  # Majority label
    X.append(window)
    y.append(1 if label == 'awake' else 0)
X, y = np.array(X), np.array(y)


In [3]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(len(features),)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
48/48 [==============================] - 2s 11ms/step - loss: 0.6851 - accuracy: 0.5668 - val_loss: 0.6896 - val_accuracy: 0.5469
Epoch 2/20
48/48 [==============================] - 0s 3ms/step - loss: 0.6826 - accuracy: 0.5668 - val_loss: 0.6879 - val_accuracy: 0.5469
Epoch 3/20
48/48 [==============================] - 0s 4ms/step - loss: 0.6820 - accuracy: 0.5668 - val_loss: 0.6874 - val_accuracy: 0.5469
Epoch 4/20
48/48 [==============================] - 0s 4ms/step - loss: 0.6812 - accuracy: 0.5668 - val_loss: 0.6863 - val_accuracy: 0.5469
Epoch 5/20
48/48 [==============================] - 0s 4ms/step - loss: 0.6805 - accuracy: 0.5668 - val_loss: 0.6858 - val_accuracy: 0.5469
Epoch 6/20
48/48 [==============================] - 0s 4ms/step - loss: 0.6797 - accuracy: 0.5668 - val_loss: 0.6850 - val_accuracy: 0.5469
Epoch 7/20
48/48 [==============================] - 0s 4ms/step - loss: 0.6786 - accuracy: 0.5668 - val_loss: 0.6836 - val_accuracy: 0.5469
Epoch 8/20
48/48 [=

In [4]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Optimize for size
tflite_model = converter.convert()

# Save the model
with open('sleep_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\nguye\AppData\Local\Temp\tmpxtrff5vt\assets


In [7]:
# Extract min and max values
min_values = scaler.data_min_
max_values = scaler.data_max_

# Print them for reference
for feature, min_val, max_val in zip(features, min_values, max_values):
    print(f"{feature}: min = {min_val}, max = {max_val}")

AccelX: min = 9.196129, max = 10.69969
AccelY: min = -0.258574, max = 0.653617
AccelZ: min = 0.998382, max = 1.668759
GyroX: min = -0.274456, max = 0.079406
GyroY: min = 0.00906, max = 0.037305
GyroZ: min = -0.055158, max = 0.002665
IR: min = 115848.0, max = 135484.0
